<a href="https://colab.research.google.com/github/kjsmi2/mediapipe/blob/main/batch_ip_sdx_juggernautl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ruamel-yaml

In [ ]:
from ruamel.yaml import YAML
import pathlib

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!pip install -q diffusers==0.22.1 insightface onnxruntime einops accelerate -q
!pip install -q git+https://github.com/tencent-ailab/IP-Adapter.git -q
!pip install -q transformers peft controlnet_aux

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
path_to_run = "/content/drive/My Drive/kjscode/execute-jug-run-portraits.yaml"

In [ ]:
import cv2
from insightface.app import FaceAnalysis
import torch

torch.backends.cuda.matmul.allow_tf32 = True
import os

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(512, 512))

/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (512, 512)


In [ ]:
import torch
from diffusers import (
    StableDiffusionXLPipeline,
    EulerAncestralDiscreteScheduler,
    AutoencoderKL, DDIMScheduler,
    StableDiffusionXLControlNetPipeline, ControlNetModel,
)
from ip_adapter.ip_adapter_faceid import IPAdapterFaceIDXL

def setup_models(mainmodels):
    device = 'cuda'
    torch.cuda.empty_cache()

    noise_scheduler = DDIMScheduler(
        num_train_timesteps=1000, # was 1000
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
    )

    # Load VAE component
    #vae = AutoencoderKL.from_pretrained(
    #    mainmodels['vae'],
    #    torch_dtype=torch.float16
   # )
    xlmodel = mainmodels['base model']
    pipe = StableDiffusionXLPipeline.from_pretrained(
        xlmodel,
        #vae=vae,
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True,
        scheduler=noise_scheduler,
        add_watermarker=False,
    )

    pipe.enable_vae_slicing()
    ip_ckpt = mainmodels['ip']

    #pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)


    ipmodel = IPAdapterFaceIDXL(pipe, ip_ckpt, device)
    return pipe, ipmodel


In [ ]:

def setup_loras(pipe, loras):
    for key, val in loras.items():
        lora_path = val['path']
        pipe.load_lora_weights(lora_path, adapter_name=key)

def read_file(path:pathlib.Path):
    yaml = YAML(typ="safe")
    return yaml.load(path)

def configure_loras(pipe, generation):
    adapt_list = []
    weight_list = []
    for name, val in generation['adapters'].items():
        adapt_list.append(name)
        weight_list.append(val)
    pipe.set_adapters(adapt_list,  adapter_weights=weight_list)

def get_reference_embeddings_image(generation):
    impath = generation["source image path"]
    image = cv2.resize(cv2.imread(impath), (712,712))#, cv2.imread(impath))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = app.get(image)
    return torch.from_numpy(faces[0].normed_embedding).unsqueeze(0).unsqueeze(0)

def run_generation(pipe, ipmodel, key, generation):
    configure_loras(pipe, generation)
    #generation['generation_args']['num_inference_steps'] = 44
    faceid_embed = get_reference_embeddings_image(generation)
    generation['generation_args']['num_samples'] = 1

    out_images = ipmodel.generate(
        **generation['generation_args'],
        faceid_embeds=faceid_embed,
    )

    for ix, m in enumerate(out_images):
        m.resize([m.width//2, m.height//2]).save(
        f"/content/drive/My Drive/diffused/{key}-{ix}b.webp", quality=80)
    return out_images[0]

In [ ]:
path_to_file = pathlib.Path(path_to_run)
runfile = read_file(path_to_file)

pipe, ipmodel = setup_models(runfile['Main Models'])
setup_loras(pipe, runfile['Loras'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:1696: FutureWarning: You are trying to load the model files of the `variant=fp16`, but no such modeling files are available.The default model files: {'vae/diffusion_pytorch_model.safetensors', 'text_encoder_2/pytorch_model.bin', 'text_encoder_2/model.safetensors', 'unet/diffusion_pytorch_model.safetensors', 'text_encoder/model.safetensors', 'unet/diffusion_pytorch_model.bin', 'text_encoder/pytorch_mo

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

The config attributes {'feature_extractor': [None, None], 'image_encoder': [None, None]} were passed to StableDiffusionXLPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'feature_extractor': [None, None], 'image_encoder': [None, None]} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
path_to_file = pathlib.Path(path_to_run)
runfile = read_file(path_to_file)
for key, generation in tqdm(runfile['Generations'].items()):
    torch.cuda.empty_cache()
    gc.collect()
    if generation.get('skip') == 'True':
        continue
    print(f"running {key}")
    res = run_generation(pipe, ipmodel, key, generation)

  0%|          | 0/49 [00:00<?, ?it/s]

running 9ff305


  0%|          | 0/27 [00:00<?, ?it/s]

  2%|▏         | 1/49 [00:38<30:31, 38.15s/it]

running 1a5d04


  0%|          | 0/27 [00:00<?, ?it/s]

  4%|▍         | 2/49 [01:15<29:38, 37.84s/it]

running 137231


  4%|▍         | 2/49 [01:17<30:12, 38.57s/it]


KeyboardInterrupt: 

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()